In [1]:
# tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(f'length of text: {len(text)}')

length of text: 1115394


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# tokenize: maps tokens to integers
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for s, i in stoi.items()}
encode = lambda sentence: [stoi[c] for c in sentence]
decode = lambda tokens: ''.join([itos[i] for i in tokens])

print(encode('hello world'))
print(decode(encode('hello world')))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


## Other popular encoders:
* [SentencePiece](https://github.com/google/sentencepiece) (Google)
* [TikToken](https://github.com/openai/tiktoken) (OpenAI)

These take more bytes per token, so less number of tokens per sentence. They
also have larger vocab size given it's trained over larger datasets.

Side note on installing packages:
* Prefer to install within uv's venv.
* Also need to hook to Jupyter kernel. This needs (a) ctrl+shift+P in VSCode to
  select python kernel to venv, and (b) in upper-right corner, change kernel to
  point to uv's venv.

In [6]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')
enc.n_vocab
print(enc.encode('hello world'))
print(enc.decode([31373, 995]))

[31373, 995]
hello world


In [12]:
import torch
import numpy as np

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

# text is a list of characters (1115394 chars)
# encode(text) is a list of ints (1115394 ints)
# torch.tensor(encode(text)) transforms into a 1-d tensor (shape == (1115394,))
assert data.numpy().shape == (1115394,)

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [15]:
# Train / validation data split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [16]:
## Transformer training
# Never feed entire corpus to transformer in one go.
# Sample random chunks to train.
# A chucnk has max length, `block_size`.
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

Taking `[18, 47, 56, 57, 58,  1, 15, 47, 58]` as an example:
* In the context of `[]`, `18` comes next.
* In the context of [`18`], `47` comes next.
* In the context of [`18`, `47`], `56` comes next.
* And so on.

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context}, target is {target}')

when input is tensor([18]), target is 47
when input is tensor([18, 47]), target is 56
when input is tensor([18, 47, 56]), target is 57
when input is tensor([18, 47, 56, 57]), target is 58
when input is tensor([18, 47, 56, 57, 58]), target is 1
when input is tensor([18, 47, 56, 57, 58,  1]), target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


We trained all O(block_size) number of samples (context length = 1 ... 8).
* This is not just for computational efficiency.
* Also useful for transformer to see context of different lengths.
* Crucial for inference, because when we start sampling, a single token can
  start the generation process because training has seen len-1 and len-n context
  windows during training. 
* After `block_size`, we'll have to start truncating, because transformer never
  sees more than `block_size` inputs when it predicts next token.


Now we introduce a new dimension: batch (in addition to time dimension).
* As we sample chunks of text, every time we feed into a transformer, we have
  multiple mini batches of multiple chunks of texts, **stacked up in a single tensor**.
* This is just for efficiency to keep GPUs busy. **Each batch is processed independently**.

In [32]:
torch.manual_seed(1337)
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # max context window.

def get_batch(split):
    # generates a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # ix is the starting index of the context window; generate `batch_size` such
    # indexes in parallel, 1 per batch.
    # ix is a 1-d tensor of `block_size` elements in 1st dimension.
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # Let's say x == [1, 2, 3], then in a single batch, we have 3 samples (aka 3
    # context windows), and for each, the expected target is the next element:
    # [1] -> 2
    # [1, 2] -> 3
    # [1, 2, 3] -> 4
    #
    # From x's perspective, [data[i:i+block_size] for i in ix] is a 2d tensor,
    # but it really contains 3-dimensional information:
    # * sampling start index, randomized via torch.randint (1st tensor dim)
    # * context window length, varies from 1 to block_size (2nd tensor dim)
    # * sub-window per context length (not explicitly a tensor dim)
    #
    # IOW, each element in the 2d x tensor is an *array* representing
    # *full* context window. This single element can be expanded to exactly
    # block_size sub context windows aka sub arrays.
    #
    # From y's perspective, it's also a 2d tensor, but each element is a scalar
    # not an array. i-th scalar in y is the target for a sub context window.
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

for b in range(batch_size):  # batch dim
    for t in range(block_size):  # time dim
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context}, output is {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is tensor([24]), output is 43
when input is tensor([24, 43]), output is 58
when input is tensor([24, 43, 58]), output is 5
when input is tensor([24, 43, 58,  5]), output is 57
when input is tensor([24, 43, 58,  5, 57]), output is 1
when input is tensor([24, 43, 58,  5, 57,  1]), output is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]), output is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), output is 39
when input is tensor([44]), output is 53
when input is tensor([44, 53]), output is 56
when input is tensor([44, 53, 56]), output is 1
when input is tensor([44, 53, 56,  1

For 1st batch `x=[24, 43, 58,  5, 57,  1, 46, 43]` and `y=[43, 58,  5, 57,  1, 46, 43, 39]`:
* input `[24]` targets `[43]`
* input `[24, 43]` targets `[58]`
* input `[24, 43, 58]` targets `[5]`
* ...
* input `[24, 43, 58,  5, 57,  1, 46, 43]` targets `[39]`

Terminologies:
* `Chunk`: largest context window with `block_size` tokens.
* `Context window`: 1 or more contigeous tokens as input.
* `Target`: a single token.
* `Batch`:  a set of chunks.

In [ ]:
# Bi-gram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a
        # lookup table.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of ints.
        logits = self.token_embedding_table(idx)  # (B, T, C), C=num channels=vocab_size
        
        # logits is one round of inference. we can then evaluate loss.
        # small quirk: pytorch wants channel dimension to come as 2nd dimension,
        # i.e. (B*T, C), not (B, T, C).
        B, T, C = logits.shape
        if targets is not None:
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        else:
            return logits, None
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indicies of the current context.
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# generate text. start with token index 0 which is newline. batch size 1.
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


IndexError: too many indices for tensor of dimension 2

Back-of-envolope calculations:
* Loss currently is about 4.88
* Vocab size is 65
* -ln(65) == 4.17 < 4.88
* This is saying our initial predictions are not super diffused, got more
  entropy, so we're guessing wrong.